In [1]:
!pip install openai langchain
!pip install langchain_openai
!pip install pydantic

In [2]:
import os
import copy

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

azurechatmodel = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [3]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict
import json
#from pydantic import BaseModel, Field
#from typing import List


In [4]:
class Ouptut(BaseModel):
    excd: str = Field(description="excd")
    claimNotes: str = Field(description="claim Notes")

class Claim(BaseModel):
    claimId: str = Field(description="claimId")
    # excd: str = Field(description="excd")
    billType : int = Field (description="billType")
    reimbursementMethod : int = Field (description="reimbursement Method")
    duplicate :bool = Field(description="duplicate")
    denyReason : str = Field (description="Deny reason")
    reimbursementMethod : str = Field (description="Reimbursment method")
    hasAdditionalRevCodes : bool = Field (description="has  Additional Rev Codes ")
    pending :bool = Field(description="pending")

    # excd : str = Field(description= "output or expected excd")
    # claimNotes: str  = Field(description="claim Notes")


# Define your desired data structure.
class Claims(BaseModel):
    currentClaim : Claim = Field(description=" current claim")
    previousClaim : Claim = Field(description="previous Claim  ")
    output_current : Ouptut = Field(description="Ouptut for current claim")
    # output_previous : Ouptut = Field(description="Ouptut for previous claim")

class ClaimsArr(BaseModel):
    details : list[Claims]

In [7]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = """
You are a QA analyst, your job is to generate test data for testing Drools rules.  Please use "Drool File" to arrive at the test data.  Test data should test all possible flows in "Drool File".
Generate testdata as JSON as mentioned in the "output" only and follow the "Instructions" to generate test cases.


Instructions :

test data should cover positive flows , negative flows and edge cases as well.
Give the output only in the "output format".
Give the output as one single json.
Give me 3 test cases. 

Drool File:
package com.edw.queries;

unit ClaimUnitData;

import com.edw.model.Claim;
import com.edw.model.Detail;

rule NonPaymentZeroPay when
$D: /details[currentClaim.billType == 0]
then
modify($D.currentClaim) {
    setExcd("5JU"),
    setClaimNotes("Claim submitted with bill type 0 provider does not expect payment.")
};
System.out.println("NonPaymentZeroPay "+$D);
end
"""


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ClaimsArr)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | azurechatmodel | parser

claimstestcase = chain.invoke({"query": joke_query})

In [8]:
claimstestcase

{'details': [{'currentClaim': {'claimId': '12345',
    'billType': 0,
    'reimbursementMethod': 'Check',
    'duplicate': False,
    'denyReason': '',
    'hasAdditionalRevCodes': False,
    'pending': False},
   'previousClaim': {'claimId': '54321',
    'billType': 1,
    'reimbursementMethod': 'Direct Deposit',
    'duplicate': False,
    'denyReason': '',
    'hasAdditionalRevCodes': True,
    'pending': True},
   'output_current': {'excd': '5JU',
    'claimNotes': 'Claim submitted with bill type 0 provider does not expect payment.'}},
  {'currentClaim': {'claimId': '67890',
    'billType': 2,
    'reimbursementMethod': 'Direct Deposit',
    'duplicate': True,
    'denyReason': 'Invalid documentation',
    'hasAdditionalRevCodes': False,
    'pending': False},
   'previousClaim': {'claimId': '09876',
    'billType': 1,
    'reimbursementMethod': 'Check',
    'duplicate': False,
    'denyReason': '',
    'hasAdditionalRevCodes': False,
    'pending': False},
   'output_current': {'e

In [16]:
for item in claimstestcase['details']:
    print(item)

{'currentClaim': {'claimId': '12345', 'billType': 0, 'reimbursementMethod': 'Check', 'duplicate': False, 'denyReason': '', 'hasAdditionalRevCodes': False, 'pending': False}, 'previousClaim': {'claimId': '54321', 'billType': 1, 'reimbursementMethod': 'Direct Deposit', 'duplicate': False, 'denyReason': '', 'hasAdditionalRevCodes': True, 'pending': True}, 'output_current': {'excd': '5JU', 'claimNotes': 'Claim submitted with bill type 0 provider does not expect payment.'}}
{'currentClaim': {'claimId': '67890', 'billType': 2, 'reimbursementMethod': 'Direct Deposit', 'duplicate': True, 'denyReason': 'Invalid documentation', 'hasAdditionalRevCodes': False, 'pending': False}, 'previousClaim': {'claimId': '09876', 'billType': 1, 'reimbursementMethod': 'Check', 'duplicate': False, 'denyReason': '', 'hasAdditionalRevCodes': False, 'pending': False}, 'output_current': {'excd': '', 'claimNotes': ''}}
{'currentClaim': {'claimId': '24680', 'billType': 3, 'reimbursementMethod': 'Check', 'duplicate': F

In [17]:
original_testcase = 		{
			"name": "http://localhost:8080/get-claim",
			"event": [
				{
					"listen": "prerequest",
					"script": {
						"exec": [
							"// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
						],
						"type": "text/javascript",
						"packages": {}
					}
				},
				{
					"listen": "test",
					"script": {
						"exec": [
							"pm.test(\"Status code is 200\", function () {\r",
							"    pm.response.to.have.status(200);\r",
							"});\r",
							"pm.test(\"excd\", function () {\r",
							"    var jsonData = pm.response.json();\r",
							"    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
							"});"
						],
						"type": "text/javascript",
						"packages": {}
					}
				}
			],
			"request": {
				"method": "POST",
				"header": [],
				"body": {
					"mode": "raw",
					"raw": "{\"details\":[{testclaim}]}",
					"options": {
						"raw": {
							"language": "json"
						}
					}
				},
				"url": {
					"raw": "http://localhost:8080/get-claim",
					"protocol": "http",
					"host": [
						"localhost"
					],
					"port": "8080",
					"path": [
						"get-claim"
					]
				}
			},
			"response": []
		}

In [18]:
modified_testcase = original_testcase.copy()

In [19]:
modified_testcase

{'name': 'http://localhost:8080/get-claim',
 'event': [{'listen': 'prerequest',
   'script': {'exec': ['// pm.collectionVariables.set("c_billType",data["currentClaim"]["billType"]);'],
    'type': 'text/javascript',
    'packages': {}}},
  {'listen': 'test',
   'script': {'exec': ['pm.test("Status code is 200", function () {\r',
     '    pm.response.to.have.status(200);\r',
     '});\r',
     'pm.test("excd", function () {\r',
     '    var jsonData = pm.response.json();\r',
     '    pm.expect(jsonData[0].currentClaim.excd).eq("5JU")\r',
     '});'],
    'type': 'text/javascript',
    'packages': {}}}],
 'request': {'method': 'POST',
  'header': [],
  'body': {'mode': 'raw',
   'raw': '{"details":[{testclaim}]}',
   'options': {'raw': {'language': 'json'}}},
  'url': {'raw': 'http://localhost:8080/get-claim',
   'protocol': 'http',
   'host': ['localhost'],
   'port': '8080',
   'path': ['get-claim']}},
 'response': []}

In [20]:
modified_finaltestcases = []

In [21]:
for item in claimstestcase['details']:
    # Replace {testclaim} with the current item
    modified_testcase = copy.deepcopy(original_testcase)
    modified_testcase['request']['body']['raw'] = modified_testcase['request']['body']['raw'].replace('{testclaim}', str(item))
    modified_finaltestcases.append(modified_testcase)
    #print(modified_finaltestcases)

[{'name': 'http://localhost:8080/get-claim', 'event': [{'listen': 'prerequest', 'script': {'exec': ['// pm.collectionVariables.set("c_billType",data["currentClaim"]["billType"]);'], 'type': 'text/javascript', 'packages': {}}}, {'listen': 'test', 'script': {'exec': ['pm.test("Status code is 200", function () {\r', '    pm.response.to.have.status(200);\r', '});\r', 'pm.test("excd", function () {\r', '    var jsonData = pm.response.json();\r', '    pm.expect(jsonData[0].currentClaim.excd).eq("5JU")\r', '});'], 'type': 'text/javascript', 'packages': {}}}], 'request': {'method': 'POST', 'header': [], 'body': {'mode': 'raw', 'raw': '{"details":[{\'currentClaim\': {\'claimId\': \'12345\', \'billType\': 0, \'reimbursementMethod\': \'Check\', \'duplicate\': False, \'denyReason\': \'\', \'hasAdditionalRevCodes\': False, \'pending\': False}, \'previousClaim\': {\'claimId\': \'54321\', \'billType\': 1, \'reimbursementMethod\': \'Direct Deposit\', \'duplicate\': False, \'denyReason\': \'\', \'hasAd

In [22]:
modified_finaltestcases

[{'name': 'http://localhost:8080/get-claim',
  'event': [{'listen': 'prerequest',
    'script': {'exec': ['// pm.collectionVariables.set("c_billType",data["currentClaim"]["billType"]);'],
     'type': 'text/javascript',
     'packages': {}}},
   {'listen': 'test',
    'script': {'exec': ['pm.test("Status code is 200", function () {\r',
      '    pm.response.to.have.status(200);\r',
      '});\r',
      'pm.test("excd", function () {\r',
      '    var jsonData = pm.response.json();\r',
      '    pm.expect(jsonData[0].currentClaim.excd).eq("5JU")\r',
      '});'],
     'type': 'text/javascript',
     'packages': {}}}],
  'request': {'method': 'POST',
   'header': [],
   'body': {'mode': 'raw',
    'raw': '{"details":[{\'currentClaim\': {\'claimId\': \'12345\', \'billType\': 0, \'reimbursementMethod\': \'Check\', \'duplicate\': False, \'denyReason\': \'\', \'hasAdditionalRevCodes\': False, \'pending\': False}, \'previousClaim\': {\'claimId\': \'54321\', \'billType\': 1, \'reimbursement

In [24]:
Json_body = {
	"info": {
		"_postman_id": "6259f6b2-9049-454c-95e4-b82805bfa3f8",
		"name": "testcollection",
		"schema": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json",
		"_exporter_id": "98974"
	},
	"item":{"textreplace"}

}

In [26]:
Json_body["item"] = modified_finaltestcases

In [27]:
Json_body

{'info': {'_postman_id': '6259f6b2-9049-454c-95e4-b82805bfa3f8',
  'name': 'testcollection',
  'schema': 'https://schema.getpostman.com/json/collection/v2.1.0/collection.json',
  '_exporter_id': '98974'},
 'item': [{'name': 'http://localhost:8080/get-claim',
   'event': [{'listen': 'prerequest',
     'script': {'exec': ['// pm.collectionVariables.set("c_billType",data["currentClaim"]["billType"]);'],
      'type': 'text/javascript',
      'packages': {}}},
    {'listen': 'test',
     'script': {'exec': ['pm.test("Status code is 200", function () {\r',
       '    pm.response.to.have.status(200);\r',
       '});\r',
       'pm.test("excd", function () {\r',
       '    var jsonData = pm.response.json();\r',
       '    pm.expect(jsonData[0].currentClaim.excd).eq("5JU")\r',
       '});'],
      'type': 'text/javascript',
      'packages': {}}}],
   'request': {'method': 'POST',
    'header': [],
    'body': {'mode': 'raw',
     'raw': '{"details":[{\'currentClaim\': {\'claimId\': \'12345